### 코더의 소회 :
날씨 : 오늘의 학습 < 보통☁️>했습니다. [☀️⛅☁️🌧️🌩️⛈️]
- 배운점 : 코드 단계별 생성하는 결과물에 대한 이해가 부족하다는 것을 깨달았습니다.
- 아쉬운점 : 여러 차례 다양한 시도를 했지만 오류가 오류를 낳는 결과만 초래하여 최종결과물을 못낸 것, 전날 18:00때와 큰 차이가 없는 것이 아쉽습니다. 
- 느낀점 :  어디부터 공부를 다시 해야하나 생각이 많아졌습니다. 융님께서 알려주신 포스트잇 방법으로 알고 있는 바를 정리해보아야겠습니다. 

(참고 : https://github.com/Annyong2/AIFFEL_quest_rs/tree/master/GoingDeeper/Gdr05)

### 전체 코드 실행 플로우 (목차):

**STEP 1. 불러오기 : 라이브러리 & 데이터**  

**STEP 2. 데이터 정제 및 토큰화**  

**STEP 3. 모델 설계**  

**STEP 4. 모델 훈련**  

**STEP 5. 번역 및 시각화**

---

**[TIP] STEP 4. 모델 설계**
- 한국어를 영어로 잘 번역해 줄 멋진 Attention 기반 Seq2seq 모델을 설계하세요!
- 앞서 만든 모델에 Dropout 모듈을 추가하면 성능이 더 좋아집니다!
- Embedding Size와 Hidden Size는 실험을 통해 적당한 값을 맞춰 주도록 합니다!

**[TIP] STEP 5. 모델 훈련**  
- traing loss 안정적으로 떨어지면서 학습 진행  
  
**[TIP] STEP 6. 모델 평가(테스트)**  
- 데스트용 디코더 모델의 출력 결과가 정답과 어느 정도 유사한 영번역 진행

---

# STEP 1. 불러오기 : 라이브러리 & 데이터   
---

### 1.1 라이브러리

In [171]:
import tensorflow as tf
import numpy as np
from tqdm.notebook import tqdm
import re
import os
import sentencepiece as spm
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm  # 폰트 설정


import io
import time
import random

import seaborn

In [172]:
# 한글설정
%config InlineBackend.figure_format = 'retina'

fontpath = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'
font = fm.FontProperties(fname=fontpath, size=9)
plt.rc('font', family='NanumBarunGothic')
mpl.font_manager.findfont(font)

print("완료!")

완료!


In [173]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [174]:
# 데이터셋 저장경로
# data_dir = os.getenv('HOME')+'/aiffel/transformer/data'
kor_path = '/content/drive/MyDrive/camp/korean-english-park.train.ko' # data_dir+"/korean-english-park.train.ko"
eng_path = '/content/drive/MyDrive/camp/korean-english-park.train.en' # data_dir+"/korean-english-park.train.en"


### 데이터 확인

데이터 출처 : https://github.com/jungyeul/korean-parallel-corpora/tree/master/korean-english-news-v1

#### 1.3 데이터 확인 : TRAIN

In [175]:
# KOREAN
# 데이터 형태 확인
with open(kor_path, "r") as f:
    raw_ko = f.read().splitlines()

print("Data Size:", len(raw_ko))
print("Example:")

for sen_ko in raw_ko[0:100][::20]: print(">>", sen_ko)

Data Size: 94123
Example:
>> 개인용 컴퓨터 사용의 상당 부분은 "이것보다 뛰어날 수 있느냐?"
>> 북한의 핵무기 계획을 포기하도록 하려는 압력이 거세지고 있는 가운데, 일본과 북한의 외교관들이 외교 관계를 정상화하려는 회담을 재개했다.
>> "경호 로보트가 침입자나 화재를 탐지하기 위해서 개인적으로, 그리고 전문적으로 사용되고 있습니다."
>> 수자원부 당국은 논란이 되고 있고, 막대한 비용이 드는 이 사업에 대해 내년에 건설을 시작할 계획이다.
>> 또한 근력 운동은 활발하게 걷는 것이나 최소한 20분 동안 뛰는 것과 같은 유산소 활동에서 얻는 운동 효과를 심장과 폐에 주지 않기 때문에, 연구학자들은 근력 운동이 심장에 큰 영향을 미치는지 여부에 대해 논쟁을 해왔다.


In [176]:
# ENGLISH
# 데이터 형태 확인
with open(eng_path, "r") as f:
    raw_en = f.read().splitlines()

print("Data Size:", len(raw_en))
print("Example:")

for sen_en in raw_en[0:100][::20]: print(">>", sen_en)

Data Size: 94123
Example:
>> Much of personal computing is about "can you top this?"
>> Amid mounting pressure on North Korea to abandon its nuclear weapons program Japanese and North Korean diplomats have resumed talks on normalizing diplomatic relations.
>> “Guard robots are used privately and professionally to detect intruders or fire,” Karlsson said.
>> Authorities from the Water Resources Ministry plan to begin construction next year on the controversial and hugely expensive project.
>> Researchers also have debated whether weight-training has a big impact on the heart, since it does not give the heart and lungs the kind of workout they get from aerobic activities such as brisk walking or running for at least 20 minutes.


##### 코멘트:
한국어데이터와 영어데이터 크기가 동일하다.    
두 파일로 나눠져있는 데이터들은 순서(idx)도 매핑되어있다.(raw_ko[1] = raw_en[1])      
=> 병렬로 묶는다

---

# Step 2. 데이터 정제 및 토큰화   

2.1 데이터 정제  
2.2 데이터 토큰화  

---

### 2.1.1 데이터 정제 : 병렬 데이터셋 생성 및 중복제거(set()사용)

In [177]:
def clean_corpus(kor_path, eng_path):
    with open(kor_path, "r") as f: kor = f.read().splitlines()
    with open(eng_path, "r") as f: eng = f.read().splitlines()
    assert len(kor) == len(eng)

    dataset = list(zip(kor, eng))
    print("병렬 데이터셋 생성 완료")
    cleaned_corpus = list(set(dataset))
    print(f"병렬 데이터셋 중복제거 완료 : {len(dataset)} ->{len(cleaned_corpus)}","\n")
    ko_corpus, en_corpus = zip(*cleaned_corpus)

    return cleaned_corpus, ko_corpus, en_corpus


cleaned_corpus, ko_corpus, en_corpus = clean_corpus(kor_path, eng_path)

print("Type(cleaned_corpus) : ", type(cleaned_corpus),"\n",cleaned_corpus[:1])

병렬 데이터셋 생성 완료
병렬 데이터셋 중복제거 완료 : 94123 ->78968 

Type(cleaned_corpus) :  <class 'list'> 
 [('갤럽 여론조사는 1930년대부터 실시됐지만 대통령에 대한 지지율을 월별로 발표한 것은 트루먼 시대부터다.', "While Gallup polling goes back to the 1930s, it wasn't until the Truman years that they began surveying monthly approval ratings.")]


##### 중복제거 후 코멘트 :


영한 데이터를 병렬구조로 만들고, set()매써드 사용하여 중복제거를 하였다.  

데이터 중복 제거후, 데이터는 이전의 80%(94123 ->78968)정도 남았다.

---

### 2.1.2 데이터 정제 : 데이터 전처리 (lms 4가지 반영 완료)
- 모든 입력을 소문자로 변환
- 알파벳, 문장부호, 한글만 남기고 모두 제거
- 문장부호 양옆에 공백을 추가
- 문장 앞뒤의 불필요한 공백을 제거

##### 전처리 함수 preprocess_sentence 정의

In [178]:
def preprocess_sentence(sentence):
    # 1. 소문자 변환
    sentence = sentence.lower()

    # 2. 알파벳, 문장부호, 한글 외 제거
    sentence = re.sub(r'\d', '', sentence)

    # 3. 문장부호 양옆 공백 추가
    sentence = re.sub(r'(\W)', r' \1 ', sentence)

    # 4. 문장 앞뒤 공백 제거
    sentence = sentence.strip()

    # 5. 중복된 공백 제거(NoneType object)
    sentence = re.sub(r'\s+', ' ', sentence)

    return sentence

##### 전처리 함수 preprocess_sentence작동 확인

In [179]:
# 확인
text = '미하일 카미닌 러시아 외무부 대변인은 기자회견에서 “러시아가 19일 4명의 영국 외교관을 추방한 것은 영국 정부가 취한 조치에 대한 반응이다”라고 발표했다.'
print(text)
print(preprocess_sentence(text))

미하일 카미닌 러시아 외무부 대변인은 기자회견에서 “러시아가 19일 4명의 영국 외교관을 추방한 것은 영국 정부가 취한 조치에 대한 반응이다”라고 발표했다.
미하일 카미닌 러시아 외무부 대변인은 기자회견에서 “ 러시아가 일 명의 영국 외교관을 추방한 것은 영국 정부가 취한 조치에 대한 반응이다 ” 라고 발표했다 .


### 2.2.1 데이터 토큰화

##### 토큰화 함수 generate_tokenizer 정의

In [180]:
# Sentencepiece를 활용하여 학습한 tokenizer를 생성합니다.
# 코퍼스를 파일로 저장하고 SentencePiece에서 처리하도록 수정
def save_corpus(corpus, filepath):
    with open(filepath, 'w') as f:
        for line in corpus:
            f.write(f"{line}\n")

# SentencePiece에서 파일을 입력으로 사용
def generate_tokenizer(corpus,
                       vocab_size,
                       lang="ko",
                       pad_id=0,
                       bos_id=1,
                       eos_id=2,
                       unk_id=3):
    corpus_file = f"{lang}_corpus.txt"
    save_corpus(corpus, corpus_file)

    # SentencePiece 학습 시 병렬 처리 옵션 추가
    spm.SentencePieceTrainer.train(
        input=corpus_file,
        model_prefix=f"{lang}_tokenizer",
        vocab_size=vocab_size,
        pad_id=pad_id,
        bos_id=bos_id,
        eos_id=eos_id,
        unk_id=unk_id,
        num_threads=4  # 병렬 처리 옵션 추가
    )

    # 학습된 모델 불러오기
    tokenizer = spm.SentencePieceProcessor()
    tokenizer.Load(f"{lang}_tokenizer.model")

    return tokenizer

##### 전처리하기

In [181]:
SRC_VOCAB_SIZE = TGT_VOCAB_SIZE = 20000

N = len(cleaned_corpus)
kor_corpus = []
eng_corpus = []
for i in range(N):
    kor_corpus.append(preprocess_sentence(ko_corpus[i]))
    eng_corpus.append(preprocess_sentence(en_corpus[i]))

In [182]:
ko_tokenizer = generate_tokenizer(kor_corpus, SRC_VOCAB_SIZE, "ko")

In [183]:
en_tokenizer = generate_tokenizer(eng_corpus, TGT_VOCAB_SIZE, "en")

In [184]:
en_tokenizer.set_encode_extra_options("bos:eos")

True

In [185]:
src_corpus = []
tgt_corpus = []

assert len(kor_corpus) == len(eng_corpus)

# 토큰의 길이가 50 이하인 문장만 남깁니다.
for idx in tqdm(range(len(kor_corpus))):
    if len(kor_corpus[idx]) <= 50 and len(eng_corpus[idx]) <= 50:  # len(kor_corpus[idx]) <= 50:
        src_corpus.append(kor_corpus[idx])
        tgt_corpus.append(eng_corpus[idx])
    else :
        pass

print(len(src_corpus))
print(len(tgt_corpus))

  0%|          | 0/78968 [00:00<?, ?it/s]

4794
4794


In [186]:
# ko_tokenizer와 en_tokenizer를 사용하여 텍스트를 정수 시퀀스로 변환
src_corpus_tokenized = [ko_tokenizer.encode_as_ids(sentence) for sentence in src_corpus]
tgt_corpus_tokenized = [en_tokenizer.encode_as_ids(sentence) for sentence in tgt_corpus]

# 패딩처리를 완료하여 학습용 데이터를 완성합니다.
enc_train = tf.keras.preprocessing.sequence.pad_sequences(src_corpus_tokenized, padding='post')
dec_train = tf.keras.preprocessing.sequence.pad_sequences(tgt_corpus_tokenized, padding='post')

👁️👁️**코멘트 :**

---

# STEP 3. 모델 설계  
---

In [224]:
## PositionalEncoding
## 용도 : 입력 위치에 따라 위치 인코딩 값을 계산하여 반환합니다.
def positional_encoding(pos, d_model):
    def cal_angle(position, i):
        return position / np.power(10000, int(i) / d_model)

    def get_posi_angle_vec(position):
        return [cal_angle(position, i) for i in range(d_model)]

    sinusoid_table = np.array([get_posi_angle_vec(pos_i) for pos_i in range(pos)])
    sinusoid_table[:, 0::2] = np.sin(sinusoid_table[:, 0::2])
    sinusoid_table[:, 1::2] = np.cos(sinusoid_table[:, 1::2])
    return sinusoid_table

print("class positional_encoding, return sinusoid_table")

class positional_encoding, return sinusoid_table


In [262]:
## MultiHeadAttention
## 용도 : 입력에 대한 여러 헤드의 어텐션을 계산하여 결합합니다.
class MultiHeadAttention(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        self.num_heads = num_heads
        self.d_model = d_model

        self.depth = d_model // self.num_heads

        self.W_q = tf.keras.layers.Dense(d_model)
        self.W_k = tf.keras.layers.Dense(d_model)
        self.W_v = tf.keras.layers.Dense(d_model)

        self.linear = tf.keras.layers.Dense(d_model)
    '''
    def scaled_dot_product_attention(self, Q, K, V, mask):
        d_k = tf.cast(K.shape[-1], tf.float32)
        QK = tf.matmul(Q, K, transpose_b=True)

        scaled_qk = QK / tf.math.sqrt(d_k)

        if mask is not None: scaled_qk += (mask * -1e9)

        attentions = tf.nn.softmax(scaled_qk, axis=-1)
        out = tf.matmul(attentions, V)

        return out, attentions
    '''
    def scaled_dot_product_attention(self, Q, K, V, mask):
        d_k = tf.cast(K.shape[-1], tf.float32)
        QK = tf.matmul(Q, K, transpose_b=True)  # (batch_size, num_heads, seq_len_q, seq_len_k)

        # QK 차원에 맞게 마스크의 마지막 차원을 seq_len_k로 확장
        if mask is not None:
            mask = tf.expand_dims(mask, axis=2)  # (batch_size, num_heads, 1, seq_len_k)
            scaled_qk = QK / tf.math.sqrt(d_k)
            scaled_qk += (mask * -1e9)  # broadcasting으로 적용
        else:
            scaled_qk = QK / tf.math.sqrt(d_k)

        attentions = tf.nn.softmax(scaled_qk, axis=-1)
        out = tf.matmul(attentions, V)

        return out, attentions


    def split_heads(self, x):
        batch_size = x.shape[0]
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.depth))
        split_x = tf.transpose(x, perm=[0, 2, 1, 3])

        return split_x

    def combine_heads(self, x):
        batch_size = x.shape[0]
        combined_x = tf.transpose(x, perm=[0, 2, 1, 3])
        combined_x = tf.reshape(combined_x, (batch_size, -1, self.d_model))

        return combined_x


    def call(self, Q, K, V, mask):
        WQ = self.W_q(Q)
        WK = self.W_k(K)
        WV = self.W_v(V)

        WQ_splits = self.split_heads(WQ)
        WK_splits = self.split_heads(WK)
        WV_splits = self.split_heads(WV)

        out, attention_weights = self.scaled_dot_product_attention(
            WQ_splits, WK_splits, WV_splits, mask)

        out = self.combine_heads(out)
        out = self.linear(out)

        return out, attention_weights

print("class MultiHeadAttention")

class MultiHeadAttention


In [263]:
class PoswiseFeedForwardNet(tf.keras.layers.Layer):
    def __init__(self, d_model, d_ff):
        super(PoswiseFeedForwardNet, self).__init__()
        self.w_1 = tf.keras.layers.Dense(d_ff, activation='relu')
        self.w_2 = tf.keras.layers.Dense(d_model)

    def call(self, x):
        out = self.w_1(x)
        out = self.w_2(out)

        return out

print("class PoswiseFeedForwardNet")

class PoswiseFeedForwardNet


In [290]:
## EncoderLayer
## 용도 : 입력을 인코더에 전달하여 어텐션 및 피드포워드 네트워크를 적용합니다.
class EncoderLayer(tf.keras.layers.Layer):
    def __init__(self, d_model, n_heads, d_ff, dropout):
        super(EncoderLayer, self).__init__()

        self.enc_self_attn = MultiHeadAttention(d_model, n_heads)
        self.ffn = PoswiseFeedForwardNet(d_model, d_ff)

        self.norm_1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.norm_2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

        self.dropout = tf.keras.layers.Dropout(dropout)

    def call(self, x, mask):

        """
        Multi-Head Attention
        """
        residual = x
        out = self.norm_1(x)
        out, enc_attn = self.enc_self_attn(out, out, out, mask)
        out = self.dropout(out)
        out += residual

        """
        Position-Wise Feed Forward Network
        """
        residual = out
        out = self.norm_2(out)
        out = self.ffn(out)
        out = self.dropout(out)
        out += residual

        return out, enc_attn

print("class EncoderLayer, return out, enc_attn")

## DecoderLayer
## 용도 : 입력을 디코더에 전달하여 어텐션 및 피드포워드 네트워크를 적용합니다.
class DecoderLayer(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads, d_ff, dropout):
        super(DecoderLayer, self).__init__()

        self.dec_self_attn = MultiHeadAttention(d_model, num_heads)
        self.enc_dec_attn = MultiHeadAttention(d_model, num_heads)

        self.ffn = PoswiseFeedForwardNet(d_model, d_ff)

        self.norm_1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.norm_2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.norm_3 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

        self.dropout = tf.keras.layers.Dropout(dropout)

    def call(self, x, enc_out, causality_mask, padding_mask):

        """
        Masked Multi-Head Attention
        """
        residual = x
        out = self.norm_1(x)
        out, dec_attn = self.dec_self_attn(out, out, out, causality_mask) # padding_mask)
        out = self.dropout(out)
        out += residual

        """
        Multi-Head Attention
        """
        residual = out
        out = self.norm_2(out)
        out, dec_enc_attn = self.enc_dec_attn(out, enc_out, enc_out, padding_mask) # causality_mask)
        out = self.dropout(out)
        out += residual

        """
        Position-Wise Feed Forward Network
        """
        residual = out
        out = self.norm_3(out)
        out = self.ffn(out)
        out = self.dropout(out)
        out += residual

        return out, dec_attn, dec_enc_attn
print("class DecoderLayer, return out, dec_attn, dec_enc_attn")

## Encoder
## 용도 : 여러 개의 인코더 레이어를 쌓아 전체 인코더를 구성합니다.
class Encoder(tf.keras.Model):
    def __init__(self,
                 n_layers,
                 d_model,
                 n_heads,
                 d_ff,
                 dropout):
        super(Encoder, self).__init__()
        self.n_layers = n_layers
        self.enc_layers = [EncoderLayer(d_model, n_heads, d_ff, dropout)
                        for _ in range(n_layers)]

    def call(self, x, mask):
        out = x

        enc_attns = list()
        for i in range(self.n_layers):
            out, enc_attn = self.enc_layers[i](out, mask)
            enc_attns.append(enc_attn)

        return out, enc_attns

print("class Encoder, return out, enc_attns")

## Decoder
## 용도 : 여러 개의 디코더 레이어를 쌓아 전체 디코더를 구성합니다.
class Decoder(tf.keras.Model):
    def __init__(self,
                 n_layers,
                 d_model,
                 n_heads,
                 d_ff,
                 dropout):
        super(Decoder, self).__init__()
        self.n_layers = n_layers
        self.dec_layers = [DecoderLayer(d_model, n_heads, d_ff, dropout)
                            for _ in range(n_layers)]


    def call(self, x, enc_out, causality_mask, padding_mask):
        out = x

        dec_attns = list()
        dec_enc_attns = list()
        for i in range(self.n_layers):
            out, dec_attn, dec_enc_attn = \
            self.dec_layers[i](out, enc_out, causality_mask, padding_mask)

            dec_attns.append(dec_attn)
            dec_enc_attns.append(dec_enc_attn)

        return out, dec_attns, dec_enc_attns

print("class Decoder, return out, dec_attns, dec_enc_attns")

class EncoderLayer, return out, enc_attn
class DecoderLayer, return out, dec_attn, dec_enc_attn
class Encoder, return out, enc_attns
class Decoder, return out, dec_attns, dec_enc_attns


In [291]:
## Transformer
## 용도 : 인코더와 디코더를 결합하여 전체 트랜스포머 모델을 구성합니다.
class Transformer(tf.keras.Model):
    def __init__(self,
                 n_layers,
                 d_model,
                 n_heads,
                 d_ff,
                 src_vocab_size,
                 tgt_vocab_size,
                 pos_len,
                 dropout=0.2,
                 shared=False):
        super(Transformer, self).__init__()
        self.d_model = tf.cast(d_model, tf.float32)
        self.shared = shared

        # 1. Embedding Layer 정의
        self.src_embedding = tf.keras.layers.Embedding(src_vocab_size, d_model)
        self.tgt_embedding = tf.keras.layers.Embedding(tgt_vocab_size, d_model)

        # 2. Positional Encoding 정의
        self.pos_encoding = positional_encoding(pos_len, d_model)
        # 6. Dropout 정의
        self.dropout = tf.keras.layers.Dropout(dropout)

        # 3. Encoder / Decoder 정의
        self.encoder = Encoder(n_layers, d_model, n_heads, d_ff, dropout)
        self.decoder = Decoder(n_layers, d_model, n_heads, d_ff, dropout)
        # 4. Output Linear 정의
        self.out_linear = tf.keras.layers.Dense(tgt_vocab_size)

        self.shared = shared

        # 5. Shared Weights
        if shared: self.out_linear.set_weights(tf.transpose(self.tgt_embedding.weights))


    def embedding(self, emb, x):
        seq_len = x.shape[1]
        out = emb(x)

        if self.shared: out *= tf.math.sqrt(self.d_model)

        out += self.pos_encoding[np.newaxis, ...][:, :seq_len, :]
        out = self.dropout(out)

        return out

    def call(self, enc_in, dec_in, enc_mask, causality_mask, dec_mask):
        """
        순서에 따라 Encoder, Decoder, 그리고 Output 계산
        """
        enc_in = self.embedding(self.src_embedding, enc_in)
        dec_in = self.embedding(self.tgt_embedding, dec_in)

        enc_out, enc_attns = self.encoder(enc_in, enc_mask)

        dec_out, dec_attns, dec_enc_attns = \
        self.decoder(dec_in, enc_out, causality_mask, dec_mask)

        # Step 4: Out Linear(dec_out)
        logits = self.out_linear(dec_out)

        return logits, enc_attns, dec_attns, dec_enc_attns

print("Transformer")

Transformer


---

# STEP 4. 모델 훈련
___

In [292]:
def generate_padding_mask(seq):
    seq = tf.cast(tf.math.equal(seq, 0), tf.float32)
    return seq[:, tf.newaxis, tf.newaxis, :]
'''
def generate_causality_mask(size):
  mask = 1 - tf.linalg.band_part(tf.ones((size, size)), -1, 0)
  return mask  # (seq_len, seq_len)
'''
# generate_causality_mask 함수 수정
def generate_causality_mask(tgt):
    size = tf.shape(tgt)[1]  # 시퀀스 길이 추출
    mask = 1 - tf.linalg.band_part(tf.ones((size, size)), -1, 0)
    return mask

def generate_masks(src, tgt):
    enc_mask = generate_padding_mask(src)
    dec_mask = generate_causality_mask(tgt)
    dec_enc_mask = generate_padding_mask(tgt)

    # 소스 시퀀스와 타겟 시퀀스의 길이를 맞추는 방식 적용
    # tf.maximum을 사용하여 패딩 값이 음수가 되는 것을 방지
    if tf.shape(src)[1] != tf.shape(tgt)[1]:
        padding_value = tf.maximum(0, tf.shape(src)[1] - tf.shape(tgt)[1])
        dec_enc_mask = tf.pad(dec_enc_mask, [[0, 0], [0, 0], [0, 0], [0, padding_value]])

    return enc_mask, dec_enc_mask, dec_mask

In [293]:
class LearningRateScheduler(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, d_model, warmup_steps=4000):
        super(LearningRateScheduler, self).__init__()
        self.d_model = d_model
        self.warmup_steps = warmup_steps

    def __call__(self, step):
        arg1 = step ** -0.5
        arg2 = step * (self.warmup_steps ** -1.5)

        return (self.d_model ** -0.5) * tf.math.minimum(arg1, arg2)


In [294]:
transformer = Transformer(n_layers=2,
                          d_model=512,
                          n_heads=8,
                          d_ff=2048,
                          src_vocab_size=SRC_VOCAB_SIZE,
                          tgt_vocab_size=TGT_VOCAB_SIZE,
                          pos_len=50,
                          shared=True
                         )

In [295]:
learning_rate = LearningRateScheduler(512)
optimizer = tf.keras.optimizers.Adam(learning_rate,
                                     beta_1=0.9,
                                     beta_2=0.98,
                                     epsilon=1e-9)

In [296]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object(real, pred)

    # Masking 되지 않은 입력의 개수로 Scaling하는 과정
    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask

    return tf.reduce_sum(loss_)/tf.reduce_sum(mask)

In [297]:
@tf.function()
def train_step(src, tgt, model, optimizer):
    # 입력 텐서의 차원을 디버깅
    print(f"Source input shape: {src.shape}, Target input shape: {tgt.shape}")

    gold = tgt[:, 1:]
    loss = 0

    enc_mask, dec_enc_mask, dec_mask = generate_masks(src, tgt)

    # 계산된 loss에 tf.GradientTape()를 적용해 학습을 진행합니다.
    with tf.GradientTape() as tape:
        predictions, enc_attns, dec_attns, dec_enc_attns = \
        model(src, tgt, enc_mask, dec_enc_mask, dec_mask)
        loss = loss_function(gold, predictions[:, :-1])

    variables = model.trainable_variables
    gradients = tape.gradient(loss, variables)
    optimizer.apply_gradients(zip(gradients, variables))

    return loss, enc_attns, dec_attns, dec_enc_attns

---

# STEP 5. 번역 및 시각화  
---

In [298]:
# 번역 생성 함수

def evaluate(sentence, model, src_tokenizer, tgt_tokenizer):
    sentence = preprocess_sentence(sentence)

    pieces = src_tokenizer.encode_as_pieces(sentence)
    tokens = src_tokenizer.encode_as_ids(sentence)

    _input = tf.keras.preprocessing.sequence.pad_sequences([tokens],
                                                           maxlen=enc_train.shape[-1],
                                                           padding='post')

    ids = []
    output = tf.expand_dims([tgt_tokenizer.bos_id()], 0)
    for i in range(dec_train.shape[-1]):
        enc_padding_mask, combined_mask, dec_padding_mask = \
        generate_masks(_input, output)

        predictions, enc_attns, dec_attns, dec_enc_attns =\
        model(_input,
              output,
              enc_padding_mask,
              combined_mask,
              dec_padding_mask)

        predicted_id = \
        tf.argmax(tf.math.softmax(predictions, axis=-1)[0, -1]).numpy().item()

        if tgt_tokenizer.eos_id() == predicted_id:
            result = tgt_tokenizer.decode_ids(ids)
            return pieces, result, enc_attns, dec_attns, dec_enc_attns

        ids.append(predicted_id)
        output = tf.concat([output, tf.expand_dims([predicted_id], 0)], axis=-1)

    result = tgt_tokenizer.decode_ids(ids)

    return pieces, result, enc_attns, dec_attns, dec_enc_attns

In [299]:
# Attention 시각화 함수

def visualize_attention(src, tgt, enc_attns, dec_attns, dec_enc_attns):
    def draw(data, ax, x="auto", y="auto"):
        import seaborn
        seaborn.heatmap(data,
                        square=True,
                        vmin=0.0, vmax=1.0,
                        cbar=False, ax=ax,
                        xticklabels=x,
                        yticklabels=y)

    for layer in range(0, 2, 1):
        fig, axs = plt.subplots(1, 4, figsize=(20, 10))
        print("Encoder Layer", layer + 1)
        for h in range(4):
            draw(enc_attns[layer][0, h, :len(src), :len(src)], axs[h], src, src)
        plt.show()

    for layer in range(0, 2, 1):
        fig, axs = plt.subplots(1, 4, figsize=(20, 10))
        print("Decoder Self Layer", layer+1)
        for h in range(4):
            draw(dec_attns[layer][0, h, :len(tgt), :len(tgt)], axs[h], tgt, tgt)
        plt.show()

        print("Decoder Src Layer", layer+1)
        fig, axs = plt.subplots(1, 4, figsize=(20, 10))
        for h in range(4):
            draw(dec_enc_attns[layer][0, h, :len(tgt), :len(src)], axs[h], src, tgt)
        plt.show()

In [300]:
# 번역 생성 및 Attention 시각화 결합

def translate(sentence, model, src_tokenizer, tgt_tokenizer, plot_attention=False):
    pieces, result, enc_attns, dec_attns, dec_enc_attns = \
    evaluate(sentence, model, src_tokenizer, tgt_tokenizer)

    print('Input: %s' % (sentence))
    print('Predicted translation: {}'.format(result))

    if plot_attention:
        visualize_attention(pieces, result.split(), enc_attns, dec_attns, dec_enc_attns)

In [302]:
EPOCHS = 5  # 학습할 Epoch 수
BATCH_SIZE = 64  # 배치 크기


model_list =[]

examples = [
            "오바마는 대통령이다.",
            "시민들은 도시 속에 산다.",
            "커피는 필요 없다.",
            "일곱 명의 사망자가 발생했다."
]

for epoch in range(EPOCHS):
    total_loss = 0

    idx_list = list(range(0, enc_train.shape[0], BATCH_SIZE))
    random.shuffle(idx_list)
    t = tqdm(idx_list)

    for (batch, idx) in enumerate(t):
        batch_loss, enc_attns, dec_attns, dec_enc_attns = \
        train_step(enc_train[idx:idx+BATCH_SIZE],
                    dec_train[idx:idx+BATCH_SIZE],
                    transformer,
                    optimizer)

        total_loss += batch_loss

        t.set_description_str('Epoch %2d' % (epoch + 1))
        t.set_postfix_str('Loss %.4f' % (total_loss.numpy() / (batch + 1)))

    for example in tqdm(examples):
        translate(example, transformer, ko_tokenizer, en_tokenizer)

    model_list.append(transformer.get_weights())

'''
example_sentences = [
    "오바마는 대통령이다."
    "시민들은 도시 속에 산다."
    "커피는 필요 없다."
    "일곱 명의 사망자가 발생했다."
]

for epoch in range(EPOCHS):
    total_loss = 0.0  # total_loss를 부동 소수점으로 초기화
    idx_list = list(range(0, enc_train.shape[0], BATCH_SIZE))
    random.shuffle(idx_list)
    t = tqdm(idx_list)    # tqdm 진행 바 사용

    for (batch, idx) in enumerate(t):
        # 학습 단계 실행
        # train_step에서 반환된 딕셔너리를 unpacking하도록 코드를 변경했습니다.
        result = train_step(src=enc_train[idx:idx+BATCH_SIZE],
                            tgt=dec_train[idx:idx+BATCH_SIZE],
                            model=transformer,
                            optimizer=optimizer)
        if result is not None:
            batch_loss, enc_attns, dec_attns, dec_enc_attns = result['loss'], result['enc_attns'], result['dec_attns'], result['dec_enc_attns']
            total_loss += batch_loss.numpy() if hasattr(batch_loss, 'numpy') else batch_loss # batch_loss가 Tensor일 경우 NumPy 스칼라로 변환
        else:
            print(f"Warning: train_step returned None for batch {batch}, idx {idx}")
            # 이 경우 처리 방안을 고려하세요. 예: 배치 건너뛰기 또는 예외 발생

        t.set_description_str('Epoch %2d' % (epoch + 1))
        t.set_postfix_str('Loss %.4f' % (total_loss / (batch + 1))) # total_loss가 이제 부동 소수점이므로 .numpy() 제거

    for example_sentence in tqdm(example_sentences): # example_sentences를 example_sentence로 변경하여 리스트의 각 문장을 반복
        translate(example_sentence, transformer, ko_tokenizer, en_tokenizer, plot_attention=True)
'''

  0%|          | 0/75 [00:00<?, ?it/s]

TypeError: cannot unpack non-iterable NoneType object

**결론:**
